# Naive Bayes: Predicting survival from titanic crash

In [42]:
import pandas as pd

### Reading data

In [43]:
df = pd.read_csv("https://raw.githubusercontent.com/charankamarapu/ML-Naive-Bayes-Algorithm-and-Models/main/GuassianNB-predict_titanic_survival/titanic.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


### drop unnessecary data

In [44]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


### divide data into training data and training labels

In [45]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

### convert category features into numerical

In [46]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [47]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


**I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female**

In [48]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


### check for null values

In [49]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [50]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [51]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


### Model

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [53]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [54]:
model.fit(X_train,y_train)

GaussianNB()

In [55]:
model.score(X_test,y_test)

0.75

In [56]:
X_test[0:10]

,Pclass,Age,Fare,female
862,1,48.000000,25.9292,1
198,3,29.699118,7.7500,1
40,3,40.000000,9.4750,1
791,2,16.000000,26.0000,0
663,3,36.000000,7.4958,0
216,3,27.000000,7.9250,1
875,3,15.000000,7.2250,1
586,2,47.000000,15.0000,0
702,3,18.000000,14.4542,1
325,1,36.000000,135.6333,1


In [57]:
y_test[0:10]

862    1
198    1
40     0
791    0
663    0
216    1
875    1
586    0
702    0
325    1
Name: Survived, dtype: int64

In [58]:
model.predict(X_test[0:10])

array([1, 1, 1, 0, 0, 1, 1, 0, 1, 1], dtype=int64)

In [59]:
model.predict_proba(X_test[:10])

array([[7.86565203e-02, 9.21343480e-01],
       [3.91952052e-01, 6.08047948e-01],
       [4.16413373e-01, 5.83586627e-01],
       [9.12336502e-01, 8.76634981e-02],
       [9.69804699e-01, 3.01953013e-02],
       [3.84000123e-01, 6.15999877e-01],
       [3.36358488e-01, 6.63641512e-01],
       [9.38737682e-01, 6.12623176e-02],
       [3.54579423e-01, 6.45420577e-01],
       [3.33402187e-04, 9.99666598e-01]])

**Calculate the score using cross validation**

In [60]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.848     , 0.768     , 0.816     , 0.75      , 0.77419355])